#### Import Libraries

In [51]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from skimage.feature import hog, local_binary_pattern, graycomatrix, graycoprops
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from scipy.stats import skew, kurtosis
import os

In [52]:
train_path = './RetinalOCT_Dataset/train/'
test_path = './RetinalOCT_Dataset/test/'
val_path = './RetinalOCT_Dataset/val/'
classes = ['AMD','CNV','CSR','DME','DR','DRUSEN','MH','NORMAL']

#### Load Dataset

In [28]:
def load_dataset(base_path):
    images = []
    labels = []
    for label, class_name in enumerate(classes):
        folder_path = os.path.join(base_path, class_name)
        for img in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (256, 256))
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

In [29]:
train_images, train_labels = load_dataset(train_path)
val_images, val_labels = load_dataset(val_path)
test_images, test_labels = load_dataset(test_path)

In [54]:
train_data = np.load('train.npz')
train_images = train_data['images']
train_labels = train_data['labels']

test_data = np.load('test.npz')
test_images = test_data['images']
test_labels = test_data['labels']

val_data = np.load('val.npz')
val_images = val_data['images']
val_labels = val_data['labels']

In [55]:
print("Train: ", len(train_images))
print("Val: ", len(val_images))
print("Test: ", len(test_images))

Train:  18400
Val:  2800
Test:  2800


#### Feature Extraction

In [56]:
LBP_RADIUS = 2
LBP_POINTS = 8 * LBP_RADIUS
GABOR_ORIENTS = 4
IMG_SIZE = (256, 256)

In [66]:
def extract_features(img):
    if img.dtype != np.uint8:
        img_u8 = (img*255).astype(np.uint8)
    else:
        img_u8 = img.copy()

    if len(img_u8.shape) == 3:
        img_gray = cv2.cvtColor(img_u8, cv2.COLOR_BGR2GRAY)
    else:
        img_gray = img_u8

    denoised = cv2.fastNlMeansDenoising(img_gray, h=10)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(denoised)
    norm = cv2.normalize(enhanced, None, 0, 1, cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    mean_i = np.mean(norm)
    std_i = np.std(norm)
    skew_i = skew(norm.ravel())
    kurt_i = kurtosis(norm.ravel())
    p10, p50, p90 = np.percentile(norm, [10,50,90])

    sobelx = cv2.Sobel(norm, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(norm, cv2.CV_64F, 0, 1, ksize=3)
    grad_mag = np.sqrt(sobelx**2 + sobely**2)
    sobel_mean = np.mean(grad_mag)
    sobel_std = np.std(grad_mag)
    edges = cv2.Canny((norm*255).astype(np.uint8), 50,150)
    edge_density = np.sum(edges > 0)/edges.size

    lbp = local_binary_pattern((norm*255).astype(np.uint8), LBP_POINTS, LBP_RADIUS, method='uniform')
    n_bins = LBP_POINTS + 2
    (hist, _) = np.histogram(lbp.ravel(), bins = n_bins, range=(0, n_bins))
    hist = hist.astype("float")
    hist = hist/(hist.sum() + 1e-8)

    gabor_means = []
    for sigma in [2.0, 4.0, 8.0]:
        for theta in (np.linspace(0, np.pi, GABOR_ORIENTS, endpoint=False)):
            kernel = cv2.getGaborKernel((21,21), sigma=sigma, theta=theta, lambd=10.0, gamma=0.5, psi=0, ktype=cv2.CV_32F)
            filtered = cv2.filter2D(norm, cv2.CV_32F, kernel)
            gabor_means.append(np.mean(filtered))

    gabor_means = np.array(gabor_means)

    quantized = (norm * 7).astype(np.uint8)
    glcm = graycomatrix(quantized, distances=[1], angles=[0], levels=8, symmetric=True, normed=True)
    har_contrast = graycoprops(glcm, 'contrast')[0,0]
    har_energy = graycoprops(glcm, 'energy')[0,0]
    har_hom = graycoprops(glcm, 'homogeneity')[0,0]

    lap_var = cv2.Laplacian((norm * 255).astype(np.uint8), cv2.CV_64F).var()

    hog_feats = hog(norm, orientations=8, pixels_per_cell=(16,16), cells_per_block=(1,1), visualize=False, feature_vector=True)
    hog_feats = hog_feats[:50]

    orb = cv2.ORB_create(nfeatures=200)
    kps = orb.detect((norm*255).astype(np.uint8), None)
    orb_count = len(kps)

    features = np.hstack([
        sobel_mean, sobel_std, edge_density,
        mean_i, std_i, skew_i, kurt_i, p10, p50, p90,
        hist,
        gabor_means,
        har_contrast, har_energy, har_hom,
        lap_var,
        hog_feats,
        orb_count
    ])

    return features

In [67]:
def extract_features_batch(images):    
    features = []
    total = len(images)
    
    for i, img in enumerate(images, 1):
        feature = extract_features(img)
        features.append(feature)

        if i % (total//10) == 0:
            percentage = int((i / total) * 100)
            print(f"==> {percentage}% done ({i}/{total} images)\n")
    
    print("Features Extraction Completed\n")
    return np.vstack(features)

In [68]:
X_train = extract_features_batch(train_images)
X_val = extract_features_batch(val_images)
X_test = extract_features_batch(test_images)

==> 10% done (1840/18400 images)

==> 20% done (3680/18400 images)

==> 30% done (5520/18400 images)

==> 40% done (7360/18400 images)

==> 50% done (9200/18400 images)

==> 60% done (11040/18400 images)

==> 70% done (12880/18400 images)

==> 80% done (14720/18400 images)

==> 90% done (16560/18400 images)

==> 100% done (18400/18400 images)

Features Extraction Completed

==> 10% done (280/2800 images)

==> 20% done (560/2800 images)

==> 30% done (840/2800 images)

==> 40% done (1120/2800 images)

==> 50% done (1400/2800 images)

==> 60% done (1680/2800 images)

==> 70% done (1960/2800 images)

==> 80% done (2240/2800 images)

==> 90% done (2520/2800 images)

==> 100% done (2800/2800 images)

Features Extraction Completed

==> 10% done (280/2800 images)

==> 20% done (560/2800 images)

==> 30% done (840/2800 images)

==> 40% done (1120/2800 images)

==> 50% done (1400/2800 images)

==> 60% done (1680/2800 images)

==> 70% done (1960/2800 images)

==> 80% done (2240/2800 images)

==>

In [69]:
np.savez('train.npz', images = train_images, labels = train_labels)
np.savez('test.npz', images = test_images, labels = test_labels)
np.savez('val.npz', images = val_images, labels = val_labels)

np.save('X_train.npy', X_train)
np.save('X_test.npy', X_test)
np.save('X_val.npy', X_val)

In [70]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


In [71]:
svm = SVC(C=1, gamma='scale', kernel='rbf')
svm.fit(X_train_scaled, train_labels)

val_pred = svm.predict(X_val_scaled)
val_acc = np.mean(val_pred == val_labels)
print("Validation accuracy:", val_acc)

Validation accuracy: 0.6746428571428571


In [72]:
test_pred = svm.predict(X_test_scaled)
print("\n=== TEST RESULTS ===")
print(classification_report(test_labels, test_pred, target_names=classes))
print("Confusion Matrix:\n", confusion_matrix(test_labels, test_pred))


=== TEST RESULTS ===
              precision    recall  f1-score   support

         AMD       0.99      1.00      0.99       350
         CNV       0.58      0.60      0.59       350
         CSR       0.73      0.78      0.75       350
         DME       0.56      0.52      0.54       350
          DR       0.71      0.71      0.71       350
      DRUSEN       0.46      0.51      0.49       350
          MH       0.63      0.57      0.60       350
      NORMAL       0.61      0.56      0.58       350

    accuracy                           0.66      2800
   macro avg       0.66      0.66      0.66      2800
weighted avg       0.66      0.66      0.66      2800

Confusion Matrix:
 [[350   0   0   0   0   0   0   0]
 [  0 211   0  56   0  68   0  15]
 [  3   0 274   0  23   0  50   0]
 [  0  59   0 182   0  65   0  44]
 [  0   0  32   0 248   0  70   0]
 [  0  58   0  44   0 180   0  68]
 [  0   0  71   0  78   0 201   0]
 [  1  33   0  42   0  77   0 197]]
